<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/Healthwith__groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary libraries:

In [108]:
!pip install jupyter groq pillow pandas



## Initialize Required Libraries

In [109]:
import os
import json
from groq import Groq

# For handling images
from PIL import Image
import numpy as np

# For numerical data processing
import pandas as pd




## Set API Key and Model

In [110]:
api_key = "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"
client = Groq(api_key=api_key)
model = "llama3-groq-70b-8192-tool-use-preview"


## Define Functions for Multimodal Analysis
We’ll define functions to handle text analysis, numerical data processing, and image analysis.

In [139]:
def analyze_text(text: str):
    print("Analyzing text...")
    result = f"Analyzed text: {text}"
    print(result)
    return result

def process_numerical_data(data: dict):
    print("Processing numerical data...")
    df = pd.DataFrame(data)
    summary = df.describe()
    result = summary.to_json()
    print(result)
    return result

def analyze_image(image_path: str):
    print(f"Analyzing image at path: {image_path}")
    try:
        image = Image.open(image_path)
        image_array = np.array(image)
        print(f"Original image shape: {image_array.shape}")

        # Convert image to grayscale if it has more than 3 channels
        if image_array.shape[2] > 3:
            image = image.convert("L")
            image_array = np.array(image)
            print(f"Converted to grayscale image shape: {image_array.shape}")

        # Check if the image has 1 or 3 channels, which is typical for X-rays
        if len(image_array.shape) == 2:
            analysis_result = "The image is a grayscale X-ray."
        elif len(image_array.shape) == 3 and image_array.shape[2] in [1, 3]:
            analysis_result = "The image is a typical X-ray."
        else:
            analysis_result = "The image does not appear to be a typical X-ray."

        result = {
            "image_shape": image_array.shape,
            "analysis": analysis_result
        }
        print(result)
        return result
    except FileNotFoundError:
        return {"error": "Error: Image file not found."}
    except Exception as e:
        return {"error": f"Error: {str(e)}"}


## Step 1: Sample Data Preparation
Let's prepare sample data for the text report, numerical lab results, and the image path.

In [140]:
# Sample text report
text_report = "The patient is a 45-year-old male with a history of hypertension and diabetes. Recently experienced shortness of breath."

# Sample numerical lab results
numerical_lab_results = {
    "BloodPressure": [140],
    "HeartRate": [80],
    "Cholesterol": [220],
    "BloodSugar": [150]
}

# Sample image path
image_path = "/content/vwuwm0jl.png"


## Define Messages and Tools
We’ll prepare a multimodal query to analyze patient data, including a text report, a numerical dataset, and a medical image.

In [141]:
messages = [
    {"role": "system", "content": "You are an AI assistant that analyzes patient data."},
    {
        "role": "user",
        "content": "Analyze the following patient data: a text report, numerical lab results, and a chest X-ray image."
    },
    {
        "role": "assistant",
        "content": json.dumps({
            "text_report": text_report,
            "numerical_lab_results": numerical_lab_results,
            "image_path": image_path
        })
    }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "analyze_text",
            "description": "Analyzes the provided text report.",
            "parameters": {
                "type": "object",
                "properties": {"text": {"type": "string", "description": "The text report"}},
                "required": ["text"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "process_numerical_data",
            "description": "Processes the provided numerical lab results.",
            "parameters": {
                "type": "object",
                "properties": {"data": {"type": "object", "description": "The numerical lab results"}},
                "required": ["data"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_image",
            "description": "Analyzes the provided medical image.",
            "parameters": {
                "type": "object",
                "properties": {"image_path": {"type": "string", "description": "The path to the image"}},
                "required": ["image_path"]
            }
        }
    }
]


## Run the Completion Request

In [142]:
response = client.chat.completions.create(
    model=model, messages=messages, tools=tools, tool_choice="auto", max_tokens=4096
)

response_message = response.choices[0].message
print("Response message:", response_message)


Response message: ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_70zn', function=Function(arguments='{"text": "The patient is a 45-year-old male with a history of hypertension and diabetes. Recently experienced shortness of breath."}', name='analyze_text'), type='function'), ChatCompletionMessageToolCall(id='call_at30', function=Function(arguments='{"data": {"BloodPressure": [140], "HeartRate": [80], "Cholesterol": [220], "BloodSugar": [150]}}', name='process_numerical_data'), type='function'), ChatCompletionMessageToolCall(id='call_1k33', function=Function(arguments='{"image_path": "/content/vwuwm0jl.png"}', name='analyze_image'), type='function')])


## Process Tool Calls

In [143]:
tool_calls = response_message.tool_calls
print("Tool calls:", tool_calls)

if tool_calls:
    messages.append(
        {
            "role": "assistant",
            "tool_calls": [
                {"id": tool_call.id, "function": {"name": tool_call.function.name, "arguments": tool_call.function.arguments}, "type": tool_call.type}
                for tool_call in tool_calls
            ]
        }
    )

    available_functions = {
        "analyze_text": analyze_text,
        "process_numerical_data": process_numerical_data,
        "analyze_image": analyze_image
    }

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)

        try:
            function_response = function_to_call(**function_args)
        except FileNotFoundError as e:
            function_response = str(e)
        except Exception as e:
            function_response = f"Error: {str(e)}"

        print(f"Function {function_name} response:", function_response)
        messages.append({"role": "tool", "content": json.dumps(function_response), "tool_call_id": tool_call.id})

    response = client.chat.completions.create(
        model=model, messages=messages, tools=tools, tool_choice="auto", max_tokens=4096
    )

    print("Final response message:", response.choices[0].message.content)
else:
    print("No tool calls were returned by the model.")


Tool calls: [ChatCompletionMessageToolCall(id='call_70zn', function=Function(arguments='{"text": "The patient is a 45-year-old male with a history of hypertension and diabetes. Recently experienced shortness of breath."}', name='analyze_text'), type='function'), ChatCompletionMessageToolCall(id='call_at30', function=Function(arguments='{"data": {"BloodPressure": [140], "HeartRate": [80], "Cholesterol": [220], "BloodSugar": [150]}}', name='process_numerical_data'), type='function'), ChatCompletionMessageToolCall(id='call_1k33', function=Function(arguments='{"image_path": "/content/vwuwm0jl.png"}', name='analyze_image'), type='function')]
Analyzing text...
Analyzed text: The patient is a 45-year-old male with a history of hypertension and diabetes. Recently experienced shortness of breath.
Function analyze_text response: Analyzed text: The patient is a 45-year-old male with a history of hypertension and diabetes. Recently experienced shortness of breath.
Processing numerical data...
{"Bl

## Display Final Results

In [144]:
response = client.chat.completions.create(
    model=model, messages=messages, tools=tools, tool_choice="auto", max_tokens=4096
)

print(response.choices[0].message.content)


The analysis of the text report indicates that the patient is a 45-year-old male with a history of hypertension and diabetes and has recently experienced shortness of breath. The numerical lab results show that the patient's blood pressure, heart rate, cholesterol, and blood sugar levels are within the normal range. The chest X-ray image has been analyzed and is a grayscale X-ray with dimensions of 400x620 pixels.
